本文档用于处理IRIS

In [1]:
import sunpy
import sunpy.map
import numpy as np
from math import *
import astropy.units as u
from astropy.io import fits
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from sunpy.coordinates import frames
import matplotlib.gridspec as gridspec
from scipy.interpolate import interp1d
from astropy.coordinates import SkyCoord
from matplotlib.patches import ConnectionPatch
import os,cv2
from scipy.optimize import fmin
from scipy.ndimage import affine_transform
import shutil
from scipy.io import readsav
import glob
from matplotlib.patches import Rectangle
from scipy.ndimage import zoom

c:\Users\19671\anaconda3\envs\sun\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file_list=glob.glob(r'D:\Learning\PHD1st\magnetic_reconnecion\data\IRIS\iris_l2_20240618_163141_3602506433_raster\*.fits')
rsm=fits.open(file_list[399])

In [3]:
solar_xy=readsav(r'D:\Learning\PHD1st\magnetic_reconnecion\data_process\solar_xy.sav')

In [4]:
print(solar_xy.keys())

dict_keys(['solar_x', 'solar_y'])


In [5]:
solar_x=solar_xy['solar_x']
solar_y=solar_xy['solar_y']

In [ ]:
print('ss')

In [ ]:
print("Hello World")

In [ ]:
print("Hello World2")

In [ ]:
print("Hello World3")